In [ ]:
using DrWatson
@quickactivate "projectdir()"

In [ ]:
include(srcdir("rdpg.jl"))
import Main.rdpg
using PersistenceDiagrams, Pipe, Plots, ProgressMeter, Random, Ripserer, Statistics, StatsBase

In [ ]:
function scale_embeddings(X)
    return (X .- mean(eachrow(X))') * (X'X)^(-0.5)
end

function diagram(X, dim_max; alpha=true)
    points = tuple.(eachcol(X)...)
    dgm = ripserer(Alpha(points), dim_max=dim_max)
    return dgm
end

function bottleneck_distances(X, Y, dim_max)
    DX = diagram(X, dim_max)
    DY = diagram(Y, dim_max)
    return [Bottleneck()(DX[d], DY[d]) for d in 1:dim_max+1]
end

In [ ]:
function generate_sbm_sparse(n, k, p, r)
    f = (x, y) -> (r + p * (x == y)) * log(n) / n
    Z = rand(1:k, n)
    return rdpg.Adjacency(f, Z)
end

function generate_sbm_dense(n, k, p, r)
    f = (x, y) -> (r + p * (x == y))
    Z = rand(1:k, n)
    return rdpg.Adjacency(f, Z)
end

function simulate_one(A, d, epsilon, method)
    X, _, _ = rdpg.spectralEmbed(A, d=d + 1, scale=false)
    A_private = rdpg.edgeFlip(A, ϵ=epsilon)

    if method == :eps
        A_private = A_private .- rdpg.privacy(ϵ=epsilon)
    end

    X_private, _, _ = rdpg.spectralEmbed(A_private, d=d + 1, scale=false)

    if method == :eps
        X_private = X_private ./ (1 - 2 * rdpg.privacy(ϵ=epsilon))
    elseif method == :noeps
        X = rdpg.scale_embeddings(X)
        X_private = rdpg.scale_embeddings(X_private)
    end
    return bottleneck_distances(X, X_private, d)
end

## Illustration of differentially-private community detection using *persistent homology*

In [ ]:
# p, r = 0.8, 0.2
p, r = 20, 1
clust = 3
n = 900
ϵ = 2
theme(:dao)

In [ ]:
f = (x, y) -> ((r + p * (x == y))) * log(n) / n
Z = rand(1:clust, 900)
A = rdpg.Adjacency(f, Z)

In [ ]:
X, _, _ = rdpg.spectralEmbed(A, d=3, scale=false)
plt = @pipe [tuple(x...) for x in eachrow(X)]  |> scatter(_, groups=Z, lim = (-1,1), title="No  privacy")

In [ ]:
A_private = rdpg.edgeFlip(A, ϵ=ϵ)

A_with_eps = (A_private .- rdpg.privacy(ϵ=ϵ)) ./ (1 - 2 * rdpg.privacy(ϵ=ϵ))
X_with_eps, _ = rdpg.spectralEmbed(A_with_eps, d=3, scale=false)
plt_with_eps = @pipe [tuple(x...) for x in eachrow(X_with_eps)] |> scatter(_, groups=Z, title="ϵ=2; publicly available", lim=(-1, 1))

plot(plt, plt_with_eps, layout=(1, 2), size=(800,300))

In [ ]:
plt_standardized = @pipe [tuple(x...) for x in eachrow(StatsBase.standardize(ZScoreTransform, X, dims=1))] |> scatter(_, groups=Z, lim=(-3, 3), title="No  privacy (Normalized)")

X_without_eps, _ = rdpg.spectralEmbed(A_private, d=3, scale=false)
X_without_eps = StatsBase.standardize(ZScoreTransform, X_without_eps, dims=1)
plt_without_eps = @pipe [tuple(x...) for x in eachrow(X_without_eps)] |> scatter(_, groups=Z, title="ϵ=2; publicly not available", lim=(-3, 3))

plot(plt_standardized, plt_without_eps, layout=(1, 2), size=(800, 300))

## Comparison with [Seif at al. (2022)](https://arxiv.org/abs/2202.00636)

In the following, for a stochastic blockmodel with $C=3$ clusters, we examine the bottleneck distance as a function of the number of vertices $n$ when $\epsilon_n \asymp \log^k(n)$ for $k \in \{\frac 23, \frac 34, 1\}$. The result in [Seif at al. (2022)](https://arxiv.org/abs/2202.00636) requires that $\epsilon_n \asymp (\log(n)))$. 

For more information on differentially-private community detection using persistence diagrams, please see [the clustering Jupyter notebook](./clustering.ipynb).

In [ ]:
repeats = 10
Ks = [2 / 3, 3 / 4, 1]
# Ks_legend = ["1/2", "2/3", "1"]
# Ks_legend = ["0.33", "0.66", "0.90"]
Ks_legend = ["0.66", "0.75", "1.00"]
N = [100, 200, 400, 600, 800, 1000, 2000];

### Dense Regime

$$
\mathbb{E}(\mathbf{A}_{ij}) = \Theta(1)
$$

In [ ]:
p, r = 0.5, 0.1
clust = 3
n = length(N);

In [ ]:
results_dense = [zeros(repeats, n) for _ in 1:length(Ks)];

prog = Progress(convert(Int, n * repeats * length(Ks)))

# Random.seed!(2022)
for i in 1:n
    for j in 1:repeats
        A = generate_sbm_dense(N[i], clust, p, r)
        for k in 1:length(Ks)
        
            ϵn = 5 * log(N[i])^(Ks[k])
            error = simulate_one(A, 0, ϵn, :eps)
            results_dense[k][j, i] = error[1]
            next!(prog)
        
        end
    end
end

In [ ]:
theme(:default)
plt_dense = plot(title="ϵ=logᵏ(n)", xlabel="n", ylabel="Bottleneck Distance")
for k in 1:length(Ks)
    plot!(plt_dense, N,
        mean(results_dense[k], dims=1)',
        # ribbon=std(results_dense[k], dims=1),
        marker=:o,
        label="k=$(Ks_legend[k])",
        lw=3, fillapha=0.01,
        yformatter=identity
    )
end
plt_dense

### Sparse Regime

$$
\mathbb{E}(\mathbf{A}_{ij}) = \Theta\left(\frac{\log n}{n}\right)
$$

In [ ]:
p, r = 5, 1
clust = 3
n = length(N);

In [ ]:
results_sparse = [zeros(repeats, n) for _ in 1:length(Ks)];

prog = Progress(convert(Int, n * repeats * length(Ks)))

# Random.seed!(2022)
for i in 1:n
    for j in 1:repeats
        A = generate_sbm_sparse(N[i], clust, p, r)
        for k in 1:length(Ks)
        
            ϵn = 3 * log(N[i])^(Ks[k])
            error = simulate_one(A, 0, ϵn, :eps)
            results_sparse[k][j, i] = error[1]
            next!(prog)
        
        end
    end
end

In [ ]:
theme(:default)
plt_sparse = plot(title="ϵ=logᵏ(n)", xlabel="n", ylabel="Bottleneck Distance")
for k in 1:length(Ks)
    plot!(plt_sparse, N,
        mean(results_sparse[k], dims=1)',
        # ribbon=std(results_3[k], dims=1),
        marker=:o,
        label = "k=$(Ks_legend[k])",
        lw=3, fillapha=0.01,
    )
end
plt_sparse